# 划分训练集和测试集

同济子豪兄 https://space.bilibili.com/1900783

代码运行[云GPU平台](https://featurize.cn/?s=d7ce99f842414bfcaea5662a97581bd1)

2022-7-22

## 导入工具包

In [14]:
import os
import shutil
import random
import pandas as pd

## 获得所有类别名称

In [22]:
# 指定数据集路径
dataset_path = 'fruit17_full'

In [23]:
dataset_name = dataset_path.split('_')[0]
print('数据集', dataset_name)

数据集 fruit17


In [25]:
classes = os.listdir(dataset_path)

In [26]:
len(classes)

17

In [27]:
classes

['丝瓜',
 '人参果',
 '佛手瓜',
 '冬瓜',
 '南瓜',
 '哈密瓜',
 '木瓜',
 '甜瓜-伊丽莎白',
 '甜瓜-白',
 '甜瓜-绿',
 '甜瓜-金',
 '白兰瓜',
 '羊角蜜',
 '苦瓜',
 '西瓜',
 '西葫芦',
 '黄瓜']

## 创建训练集文件夹和测试集文件夹

In [28]:
# 创建 train 文件夹
os.mkdir(os.path.join(dataset_path, 'train'))

# 创建 test 文件夹
os.mkdir(os.path.join(dataset_path, 'val'))

# 在 train 和 test 文件夹中创建各类别子文件夹
for fruit in classes:
    os.mkdir(os.path.join(dataset_path, 'train', fruit))
    os.mkdir(os.path.join(dataset_path, 'val', fruit))

## 划分训练集、测试集，移动文件

In [29]:
test_frac = 0.2  # 测试集比例
random.seed(123) # 随机数种子，便于复现

In [30]:
df = pd.DataFrame()

print('{:^18} {:^18} {:^18}'.format('类别', '训练集数据个数', '测试集数据个数'))

for fruit in classes: # 遍历每个类别

    # 读取该类别的所有图像文件名
    old_dir = os.path.join(dataset_path, fruit)
    images_filename = os.listdir(old_dir)
    random.shuffle(images_filename) # 随机打乱

    # 划分训练集和测试集
    testset_numer = int(len(images_filename) * test_frac) # 测试集图像个数
    testset_images = images_filename[:testset_numer]      # 获取拟移动至 test 目录的测试集图像文件名
    trainset_images = images_filename[testset_numer:]     # 获取拟移动至 train 目录的训练集图像文件名

    # 移动图像至 test 目录
    for image in testset_images:
        old_img_path = os.path.join(dataset_path, fruit, image)         # 获取原始文件路径
        new_test_path = os.path.join(dataset_path, 'val', fruit, image) # 获取 test 目录的新文件路径
        shutil.move(old_img_path, new_test_path) # 移动文件

    # 移动图像至 train 目录
    for image in trainset_images:
        old_img_path = os.path.join(dataset_path, fruit, image)           # 获取原始文件路径
        new_train_path = os.path.join(dataset_path, 'train', fruit, image) # 获取 train 目录的新文件路径
        shutil.move(old_img_path, new_train_path) # 移动文件
    
    # 删除旧文件夹
    assert len(os.listdir(old_dir)) == 0 # 确保旧文件夹中的所有图像都被移动走
    shutil.rmtree(old_dir) # 删除文件夹
    
    # 工整地输出每一类别的数据个数
    print('{:^18} {:^18} {:^18}'.format(fruit, len(trainset_images), len(testset_images)))
    
    # 保存到表格中
    df = df.append({'class':fruit, 'trainset':len(trainset_images), 'testset':len(testset_images)}, ignore_index=True)

# 重命名数据集文件夹
shutil.move(dataset_path, dataset_name+'_split')

# 数据集各类别数量统计表格，导出为 csv 文件
df['total'] = df['trainset'] + df['testset']
df.to_csv('数据量统计.csv', index=False)

        类别              训练集数据个数            测试集数据个数      
        丝瓜                151                 37        
       人参果                146                 36        
       佛手瓜                129                 32        
        冬瓜                123                 30        
        南瓜                147                 36        
       哈密瓜                157                 39        
        木瓜                156                 38        
     甜瓜-伊丽莎白               75                 18        
       甜瓜-白                68                 17        
       甜瓜-绿                35                 8         
       甜瓜-金                42                 10        
       白兰瓜                103                 25        
       羊角蜜                157                 39        
        苦瓜                151                 37        
        西瓜                156                 38        
       西葫芦                136                 33        
        黄瓜                144  

In [31]:
df

class  testset  trainset  total
0        丝瓜     37.0     151.0  188.0
1       人参果     36.0     146.0  182.0
2       佛手瓜     32.0     129.0  161.0
3        冬瓜     30.0     123.0  153.0
4        南瓜     36.0     147.0  183.0
5       哈密瓜     39.0     157.0  196.0
6        木瓜     38.0     156.0  194.0
7   甜瓜-伊丽莎白     18.0      75.0   93.0
8      甜瓜-白     17.0      68.0   85.0
9      甜瓜-绿      8.0      35.0   43.0
10     甜瓜-金     10.0      42.0   52.0
11      白兰瓜     25.0     103.0  128.0
12      羊角蜜     39.0     157.0  196.0
13       苦瓜     37.0     151.0  188.0
14       西瓜     38.0     156.0  194.0
15      西葫芦     33.0     136.0  169.0
16       黄瓜     36.0     144.0  180.0

## 查看文件目录结构

In [32]:
!sudo snap install tree

tree 1.8.0+pkg-3fd6 from 林博仁(Buo-ren, Lin) (brlin) installedns


In [34]:
!tree fruit17_split -L 2

fruit17_split [error opening dir]

0 directories, 0 files


In [35]:
!ls

dataset_delete_test
fruit17_split
【A】安装配置环境.ipynb
【B1】图像采集（首选）.ipynb
【B2】图像采集（备用）.ipynb
【B3】制作图像分类数据集的注意事项.ipynb
【B4】删除多余文件.ipynb
【C1】下载Demo数据集.ipynb
【C2】统计图像尺寸、比例分布.ipynb
【C3】拍摄地点地图可视化.ipynb
【D】划分训练集测试集.ipynb
【E1】可视化文件夹中的图像.ipynb
【E2】统计各类别图像数量.ipynb
【F】训练图像分类识别模型的N种方法.ipynb
【a】安装配置环境.ipynb.amltmp
【b1】图像采集（首选）.ipynb.amltmp
【b3】制作图像分类数据集的注意事项.ipynb.amltmp
【b4】删除多余文件.ipynb.amltmp
【c1】下载demo数据集.ipynb.amltmp
【c2】统计图像尺寸、比例分布.ipynb.amltmp
【d】划分训练集测试集.ipynb.amltmp
各类别图像数量.pdf
图像尺寸分布.pdf
数据量统计.csv
